# Basic tutorial: Benchmarking + Gridsearch
#### Author: Abdul Jabbar

This short tutorial provides you with the basic functioning of benchmarking and gridsearch together.

The main steps of the tutorial are the following:
 1. creation of a datasets dictionaries
 2. creation of models dictionaries
 3. define hyperparameters
 4. run benchmark and gridsearch

In [7]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

import numpy as np

import torch
from torch import nn
from torch.utils.tensorboard import SummaryWriter

from gdeep.models import FFNet
from gdeep.data import TorchDataLoader

import optuna

# Initialize the tensorboard writer

In order to analyse the reuslts of your models, you need to start tensorboard.
On the terminal, move inside the `/example` folder. There run the following command:

```
tensorboard --logdir=runs
```

Then go [here](http://localhost:6006/) after the training to see all the visualisation results.

In [8]:
writer = SummaryWriter()

# Create your datasets dictionaries

In [9]:
from torch.utils.data.sampler import SubsetRandomSampler

dataloaders_dicts = []
dl = TorchDataLoader(name="CIFAR10")
train_indices = []
for i in range(5120):
    train_indices.append(i)

dl_tr, _ = dl.build_dataloader(batch_size=1024, sampler=SubsetRandomSampler(train_indices))

test_indices = []
for i in range(1024):
    test_indices.append(i)

dl_ts, _ = dl.build_dataloader(batch_size=1024, sampler=SubsetRandomSampler(test_indices))

temp_dict = {}
temp_dict["name"] = "CIFAR10_5000"
temp_dict["dataloaders"] = (dl_tr, dl_ts)

dataloaders_dicts.append(temp_dict)
print(temp_dict["name"], "created")

train_indices = []
for i in range(10240):
    train_indices.append(i)

dl_tr, _ = dl.build_dataloader(batch_size=1024, sampler=SubsetRandomSampler(train_indices))

test_indices = []
for i in range(2048):
    test_indices.append(i)

dl_ts, _ = dl.build_dataloader(batch_size=1024, sampler=SubsetRandomSampler(test_indices))

temp_dict = {}
temp_dict["name"] = "CIFAR10_10000"
temp_dict["dataloaders"] = (dl_tr, dl_ts)

dataloaders_dicts.append(temp_dict)
print(temp_dict["name"], "created")

Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified
CIFAR10_5000 created
Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified
CIFAR10_10000 created


## Define models dictionaries

In [10]:
import torchvision.models as models

models_dicts = []

model = nn.Sequential(models.resnet18(pretrained=True), nn.Linear(1000,10))
temp_dict = {}
temp_dict["name"] = "resnet18"
temp_dict["model"] = model

models_dicts.append(temp_dict)
print(temp_dict["name"], "created")

model = nn.Sequential(models.vgg16(pretrained=True), nn.Linear(1000,10))
temp_dict = {}
temp_dict["name"] = "vgg16"
temp_dict["model"] = model

models_dicts.append(temp_dict)
print(temp_dict["name"], "created")

resnet18 created
vgg16 created


## Define hyperparamters and their categorical, continuous ranges for gridsearch

In [11]:
from torch.optim import SGD, Adam, RMSprop

loss_fn = nn.CrossEntropyLoss()
lr_range = [0.0001, 0.01]
optimizers = [SGD, Adam, RMSprop]

## Benchmarking + Gridsearch

In [12]:
from gdeep.search.benchmark import Benchmark
from gdeep.search.gridsearch import Gridsearch

bench = Benchmark(models_dicts, dataloaders_dicts, loss_fn, writer)

search = Gridsearch(bench, "loss", n_trials=100)
search.start(optimizers, n_epochs=1, batch_size=1024, lr=lr_range)

[I 2021-08-11 19:56:51,709] A new study created in memory with name: no-name-f7d76963-ce23-4a40-b072-8b441381b53b


****************************************
Performing Gridsearch on Dataset: CIFAR10_5000, Model: resnet18
Epoch 1
-------------------------------
Training loss: 1.226654  [40/40]
Time taken for this epoch: 20s


[I 2021-08-11 19:57:13,946] Trial 0 finished with value: 1.226654052734375 and parameters: {'optimizer': <class 'torch.optim.rmsprop.RMSprop'>, 'lr': 0.00024742215257140626}. Best is trial 0 with value: 1.226654052734375.


Validation results: 
 Accuracy: 11.2%,                 Avg loss: 0.000259 

Done!
Epoch 1
-------------------------------
Training loss: 0.855987  [40/40]
Time taken for this epoch: 20s


[I 2021-08-11 19:57:35,889] Trial 1 finished with value: 0.8559866547584534 and parameters: {'optimizer': <class 'torch.optim.sgd.SGD'>, 'lr': 0.0008349780719929611}. Best is trial 1 with value: 0.8559866547584534.


...

[I 2021-08-11 19:59:26,167] Trial 6 finished with value: 2.0818769931793213 and parameters: {'optimizer': <class 'torch.optim.adam.Adam'>, 'lr': 0.0041654324606596235}. Best is trial 1 with value: 0.8559866547584534.


Validation results: 
 Accuracy: 4.1%,                 Avg loss: 0.000498 

Done!
Epoch 1
-------------------------------
Training loss: 2.115350  [40/40]
Time taken for this epoch: 20s


[I 2021-08-11 19:59:48,099] Trial 7 finished with value: 2.115349531173706 and parameters: {'optimizer': <class 'torch.optim.adam.Adam'>, 'lr': 0.00016664243756512866}. Best is trial 1 with value: 0.8559866547584534.


Validation results: 
 Accuracy: 4.3%,                 Avg loss: 0.000385 

Done!
Epoch 1
-------------------------------


KeyboardInterrupt: 